In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.2/transformers/3b-instruct/1/model.safetensors.index.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/config.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/model-00001-of-00002.safetensors
/kaggle/input/llama-3.2/transformers/3b-instruct/1/model-00002-of-00002.safetensors
/kaggle/input/llama-3.2/transformers/3b-instruct/1/README.md
/kaggle/input/llama-3.2/transformers/3b-instruct/1/USE_POLICY.md
/kaggle/input/llama-3.2/transformers/3b-instruct/1/tokenizer.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/tokenizer_config.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/LICENSE.txt
/kaggle/input/llama-3.2/transformers/3b-instruct/1/special_tokens_map.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/.gitattributes
/kaggle/input/llama-3.2/transformers/3b-instruct/1/generation_config.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/config.json
/kaggle/input/llama-3.2/transformers/1b-instruct/1/README.md
/kaggle/input/llama-3.

# Setup
Install the required dependencies for the notebook to run

In [2]:
%%capture --no-display

!pip install langchain
!pip install langchain_community
!pip install chromadb
!pip install pymupdf
!pip install PyPDF2
!pip install -U FlagEmbedding

In [3]:
# Import the required libraries
import torch
import langchain
import os
from pypdf import PdfReader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from langchain_community.llms import Ollama
from langchain.llms import HuggingFacePipeline
import numpy as np
import re
import shutil
import transformers
from transformers import AutoModel, AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import warnings
from FlagEmbedding import FlagReranker
from transformers import AutoModel, AutoTokenizer, T5ForConditionalGeneration
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [4]:
#Check if gpu is available
print ("True") if torch.cuda.is_available() else print("None")

True


In [5]:
CHROMA_PATH = "/kaggle/working/chroma"
DATA_PATH = "/kaggle/input/casml-dataset/Dataset_RAG (1)/book.pdf"


def clean_text(page_content):
    # Step 1: Remove excessive whitespace, tabs, and newlines
    cleaned_text = re.sub(r'\s+', ' ', page_content)

    # Step 2: Remove URLs or links
    cleaned_text = re.sub(r'http[s]?://\S+|www\.\S+', '', cleaned_text)

    # Step 3: Remove unwanted special characters (keep basic punctuation)
    cleaned_text = re.sub(r'[^A-Za-z0-9\s,.!?\'"-]', '', cleaned_text)

    # Step 4: Strip leading/trailing whitespace
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text


def load_documents():
    '''
        Load the documents ,
        clean the text and save the clean text in the document's page content.

        Returns: documents

    '''
    document_loader = PyPDFLoader(DATA_PATH)

    documents = document_loader.load()

    for doc in documents:
        if doc.metadata["page"] >= 645:
            doc.page_content = " "
        else:
            doc.page_content = clean_text(doc.page_content)
    return documents

def split_documents(documents):

    """
        Using simple Recursive Text Splitter we divide the document content into chunks.

        Returns: split_documents
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )
   
    return text_splitter.split_documents(documents)


# Uncomment the code to test the code
#documents = load_documents()
# chunks = split_documents(documents)


In [6]:
device = "cuda" 
model_kwargs = {"device":device}


# Set up the embeddings for the Meta LLama 3.2 1-b instruct version.
embedding_model_name = "BAAI/bge-large-en-v1.5"
# encoder = SentenceTransformer(embedding_model_name, device)
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs, 
                                  encode_kwargs={'normalize_embeddings':True})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

# Chroma DB

Setup the vector store to store the documents in form of chunks. Each chunk has an additional metadata field
which contains the page number and the unique ID. 

In [7]:
def calculate_chunk_ids_and_keyphrases(chunks):

    # This will create IDs like "path_to_book:6:2"
    # The ID has the following format: Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in tqdm(chunks, desc = "Calculating IDs:"):
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page - 12}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

        # Tried to extract the main keyphrases for each chunk content, but increases the inference time by a lot

    return chunks

def add_to_chroma(chunks):
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=embeddings
    )

    chunks_with_ids = calculate_chunk_ids_and_keyphrases(chunks)
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])

    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")
    return db

def create_database():

    documents = load_documents()
    chunks = split_documents(documents)
    db = add_to_chroma(chunks)
    
    return db
    
def clear_database():
    
    '''
        Clear the database if need a fresh start.
    '''
    
    if os.path.exists(CHROMA_PATH):
        print("True")
        shutil.rmtree(CHROMA_PATH)
chroma_db = create_database()

#clear_database()

Calculating IDs:: 100%|██████████| 2915/2915 [00:00<00:00, 700372.12it/s]


Number of existing documents in DB: 0
👉 Adding new documents: 2915


In [8]:
# chroma_db.get() # Run to check if the vector store is up.

# Querying, Retrieval and Generation

## Using an open-source re-ranker model to re-rank the retrieved documents from the vector database.

Just retrieving the documents based on the user query is not enough or effective. We use the re-ranking technique to get more relevant documents from the vector store. 

By specifying the `number_of_docs`, we retrieve ten times the `number_of_docs` based on the similarity search and then re-rank the documents based on the query.

In [9]:
# Loading the Reranking Model (Also open-sourced from Huggingface) 
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True, device=device) 

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [10]:
# Define the model path from the kaggle models imported already
model_path = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"
def load_llama_model_and_tokenizer(model_path):
    model_config = transformers.AutoConfig.from_pretrained(model_path)
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        config=model_config,
        device_map='auto',  # Automatically distribute the model across available devices
    )
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
    
    return model, tokenizer

llama3_model, llama3_tokenizer = load_llama_model_and_tokenizer(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# Create the query pipeline
query_pipeline = transformers.pipeline(
    "text-generation",
    model=llama3_model,
    tokenizer=llama3_tokenizer,
    torch_dtype=torch.float16,  # Uncomment this for 4bit, faster inference but less accuracy
    max_length=2048,  # Set the maximum length of generated text
    device_map="auto",  # Automatically distribute the pipeline across available devices
    num_return_sequences=1,# Generate only one sequence
    truncation=True,
    do_sample=True,  # Enable sampling from the probability distribution
    temperature=0.2,  # Adjust the temperature for less creative responses
    pad_token_id=llama3_tokenizer.eos_token_id  # Set pad_token_id explicitly, if not you will get a warning
)

# Using the the HuggingFacePipeline class from LangChain to wrap the pipeline
llama3_llm = HuggingFacePipeline(pipeline=query_pipeline)

# Query Transformation

Using the query directly to query the vector database may not prove to be effective as the query may not contain the specific information which may match with the documents. 

We transform the query first by generating the response for the query using the LLM, and then query the database using the new transformed query.

The new generated response, may generate more relevant documents than the simple user query. We use the same model to answer the query. 

In [12]:
def generate_hypothetical_response(query):


    message = """
You are a knowledgeable assistant generating a brief and context-rich hypothetical document based on the input query.
This document should resemble an informative and authoritative article, providing relevant details, explanations, and examples that could answer or address the query.

Instructions:

    1. Do not include follow up questions in the response.
    2. Avoid unnecessary repetition in the response.
    3. Keep the answer short and precise.

Answer the question: {query}

Answer:

"""
    

    response_text = llama3_llm.invoke(input=message.format(query=query))
    final_output = response_text.split("Answer:")[1].strip()
    
    return final_output

#response = generate_hypothetical_response("What are the contributions made by Freud in psychology?")

In [13]:
def retrieve_reranked_documents(query, chroma_db, number_of_docs, reranker):

    """
        Returns the re-ranked documents from the vector store
    """
    results = chroma_db.similarity_search(query, k = number_of_docs * 10)

    pairs = [[query, result.page_content] for result in results]
    scores = reranker.compute_score(pairs, normalize=True)

    sorted_results = sorted(zip(results, scores), key = lambda x : x[1], reverse=True)

    retrieved_documents = [(result.page_content, result.metadata["id"], score) for result, score in sorted_results[:number_of_docs]]

    return retrieved_documents

In [14]:
# # Retrieve ranked documents from ChromaDB
doc_reranked_texts_chromadb = retrieve_reranked_documents(generate_hypothetical_response("What are the contributions made by Freud in psycgology?"), chroma_db, 10, reranker)

# Access individual document texts and their relevance scores
for i, (doc_reranked_text, id, score) in enumerate(doc_reranked_texts_chromadb):
    print(f"Document {i+1} Page Number: {id.split(':')[1]} (Relevance Score: {score:.4f}):")
    print(doc_reranked_text)
    print()

Chunks:   0%|          | 0/2 [00:00<?, ?it/s]You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Chunks: 100%|██████████| 2/2 [00:05<00:00,  2.54s/it]

Document 1 Page Number: 632 (Relevance Score: 0.9541):
prospects, and limitations. American Psychologist, 65, 100126. Crawford, M.,  Marecek, J. 1989. Psychology reconstructs the female 19681988. Psychology of Women Quarterly, 13, 147165. Danziger, K. 1980. The history of introspection reconsidered. Journal of the History of the Behavioral Sciences, 16, 241262. Danziger, K. 1980. Wundt's psychological experiment in the light of his philosophy of science. Psychological Research, 42, 109122. Darwin, C. 1871. The descent of man and selection in relation to sex. John Murray. Darwin, C. 1872. The expression of the emotions in man and animals. John Murray. DeAngelis, T. 2010. Fear not. gradPSYCH Magazine, 8, 38. Department of Health and Human Services. n.d.. Projected future growth of the older population.  References 633

Document 2 Page Number: 607 (Relevance Score: 0.9451):
FIGURE 16.9 This is the famous couch in Freuds consulting room. Patients were instructed to lie comfortably on the c

# Mapping the sections and sub-section to page numbers

We map the sections and sub-sections to the page numbers, to provide relevant sections from where the context is retrieved.

In [15]:
# Creating mappings for references 
import pymupdf # PyMuPDF

def extract_toc(pdf_path):
    # Open the PDF
    doc = pymupdf.open(pdf_path)
    # Get the Table of Contents
    toc = doc.get_toc()
    # Each entry is a list: [level, title, page]
    for content in toc[2:]:
        content[2] = content[2] - 12
    return toc

def create_mapping(toc):
    
    '''
    Create a mapping between page numbers and sections for references.
    '''

    mapping = {}
    current_section = None

    for entry in toc:
        level, title, page = entry
        if level == 1:  # It's a main section
            current_section = title
            mapping[current_section] = {}
        elif level == 2 and current_section:  # It's a subsection
            if title not in ["Introduction", "Critical Thinking Questions", "Personal Application Questions", "Review Questions"]:
                mapping[current_section][title] = {"start_page": page, "end_page": None}
        elif level == 3:  # Handle deeper levels if needed
            continue  # Optional: Extend this logic for deeper subsections

    # Update end pages for subsections
    all_entries = list(toc)  # Flatten the TOC for easier traversal
    for i in range(len(all_entries) - 1):
        level, title, page = all_entries[i]
        next_level, next_title, next_page = all_entries[i + 1]

        # Only update if it's a subsection or section
        if level >= 1:
            current_section = None
            for section, subsections in mapping.items():
                if title == "Contents":
                    continue
                if title in subsections:
                    current_section = section
                    break
            if current_section and (title == "Summary" or title == "Key Terms"):
                mapping[current_section][title]["end_page"] = mapping[current_section][title]["start_page"] + 1
            
            elif current_section:
                mapping[current_section][title]["end_page"] = next_page
    # Update last subsections to stretch to the end of the document
    for section, subsections in mapping.items():
        for subsection, pages in subsections.items():
            if pages["end_page"] is None:
                pages["end_page"] = pages["start_page"] + 1 # Assign infinite if no end page exists

    return mapping

toc = extract_toc("/kaggle/input/casml-dataset/Dataset_RAG (1)/book.pdf")
mapping = create_mapping(toc)

In [16]:
# Function to query RAG and get the response 

def query_rag(query, chroma_db, number_of_docs):

    transformed_query = generate_hypothetical_response(query)
    reranked_documents = retrieve_reranked_documents(transformed_query, chroma_db, 
                                                     number_of_docs,
                                                    reranker)
    
    final_documents = [doc for doc,  _ , _ in reranked_documents]

    pages = [page.split(":")[1] for _, page, _ in reranked_documents]
    context_text = "\n\n".join(final_documents)


    # Prompt template for the LLM
    messages = f""" You are an AI assistant trained to provide clear, detailed and accurate responses. Please avoid repetition and ensure your explanation is easy to understand.
        
        Context: {context_text}
    
        Instructions:
        1. Understand all the context provided to craft your response and ensure relevance.
        2. Avoid unnecessary repetition in your response.
        4. Exclude closing phrases such as "Best regards" or "Let me know if I can help you further."
        
        Task: Respond to the following question by providing a detailed and relevant answer keeping in mind the context.

        Question: {query}
        
        Answer:

    """
    
    response_text = llama3_llm.invoke(input=messages)
    
    prompt = messages.split("Answer:")[0].strip()
    
    answer = response_text.split("Answer:")[1].strip()
    
    final_output_llm = f"{answer}"
    
    # Return the context, response, and page numbers
    return context_text, final_output_llm, pages

In [17]:
def resolve_references(mapping, pages):
    
    """
    Resolves the references for given pages in the format 
    "section/subsection" for better reference stucture.
    """
    
    references = []

    for page in pages:
        for section, subsections in mapping.items():
            for subsection, page_range in subsections.items():

                if page_range["start_page"] is None:
                    continue
                if page == page_range["start_page"]:
                    references.append(f"{section}/{subsection}")
                if page == page_range["end_page"]:
                    references.append(f"{section}/{subsection}")
                if page_range["start_page"] <= int(page) <= page_range["end_page"]:
                    references.append(f"{section}/{subsection}")
                    break  # Stop searching once matched
            else:
                continue
            break

    return references

In [18]:
import csv

def save_row_in_csv(query_id, context_text, response_text, references, csv_file):    
    """
    Save a row in the CSV file with the provided data.

    Args:
        query_id (str): Unique identifier for the query.
        context_text (str): The context text used to generate the response.
        response_text (str): The generated response text.
        references (dict): References containing sections and pages.
        csv_file (str): Path to the CSV file.

    """
    # Ensure references are converted to a string format for CSV storage
    references_str = str(references)
    assert isinstance(references, dict)

    write_header = not os.path.exists(csv_file) or os.stat(csv_file).st_size == 0
    # Open the CSV file in append mode and write the row
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write headers if needed
        if write_header:
            writer.writerow(["ID", "context", "answer", "references"])
        # Write the row data
        writer.writerow([query_id, context_text, response_text, references_str])

    print(f"Row saved successfully to {csv_file}")


# Answer the queries one by one, and save in CSV file

In [19]:
# For each query in the queries.json file, run the rag on the quey and extract the references.
import json
def answer_queries(json_file):
    """
        This function queries the RAG for each query in the json file and saves it in the csv file.

        Params: json file path

        Returns: CSV file
    """
    with open(json_file) as f:
        queries = json.load(f)
    
    for idx, query in enumerate(queries):
        print(f"Answering question number : {idx+1} \n\n ")
        query_id = query["query_id"]
        question_text = query["question"]
        context_text, response_text, pages = query_rag(question_text, chroma_db, 8)
        references = resolve_references(mapping, pages)
        references = {"sections": list(set(references)), "pages": [f"{page}" for page in pages]}
        save_row_in_csv(query_id, context_text, response_text, references, "submission.csv")
answer_queries("/kaggle/input/casml-dataset/Dataset_RAG (1)/queries.json")

Answering question number : 1 

 


Chunks: 100%|██████████| 2/2 [00:02<00:00,  1.42s/it]


Row saved successfully to submission.csv
Answering question number : 2 

 


Chunks: 100%|██████████| 2/2 [00:02<00:00,  1.27s/it]


Row saved successfully to submission.csv
Answering question number : 3 

 


...

Row saved successfully to submission.csv
Answering question number : 49 

 


Chunks: 100%|██████████| 2/2 [00:02<00:00,  1.00s/it]


Row saved successfully to submission.csv
Answering question number : 50 

 


Chunks: 100%|██████████| 2/2 [00:02<00:00,  1.39s/it]


Row saved successfully to submission.csv
